In [4]:
import pandas as pd
import numpy as np
l = 0.1

def f(x, beta):
    return beta[0] * x ** (beta[1]) * np.e ** (beta[2] * x)

def r(y, x, beta):
    res = y - beta[0] * x ** (beta[1]) * np.e ** (beta[2] * x)
    return res

def r_adj(y, x, beta):
    res = r(y, x, beta)
    if res < 0:
        res *= 0.9
    return res

def drdb(x, beta, j):
    if j == 0:
        return -x ** (beta[1]) * np.e ** (beta[2] * x)
    elif j == 1:
        return -beta[0] * np.log(x) * (x ** (beta[1])) * (np.e ** (beta[2]* x))
    elif j == 2:
        return -beta[0] * (x ** (beta[1]+ 1)) * (np.e ** (beta[2]* x))
    else:
        print("error")

def cost(ys, xs, beta):
    return sum([r_adj(ys[i], x, beta) ** 2 for i, x in enumerate(xs)])

def costd(y, x, beta):
    return np.array([r_adj(y, x, beta) * drdb(x, beta, j) for j in range(3)])

In [5]:
set1 = [150, 150.909090909091, 151.818181818182, 152.727272727273, 151.136363636364, 152.045454545455, 152.954545454545, 153.863636363636, 152.272727272727, 153.181818181818, 154.090909090909, 152.5, 153.409090909091, 154.318181818182, 155.227272727273, 146.136363636364, 153.276074700493, 154.013331445122, 158.560917921974, 159.292249954471, 159.030552209196, 159.761357745856, 160.491343993796, 161.031444520339, 161.760006904183, 162.755232002254, 157.695704865228, 158.688765798209, 158.794474873158, 155.71532630068, 156.783635334823, 157.853036709491, 158.762127618582, 159.671218527673]
set2 = [162.5, 163.484848484849, 164.469696969697, 170.454545454546, 171.439393939394, 177.424242424242, 178.409090909091, 181.89393939394, 182.878787878788, 183.863636363637, 182.348484848485, 183.333333333333, 184.318181818182, 180.30303030303, 181.287878787879, 182.272727272727, 185.027213886437, 187.646775242667, 188.421257387558, 189.085611707775, 189.858480807301, 190.630483461648, 191.5915700363, 192.361629765503, 193.130826199045, 194.218513249495, 195.306779473028, 196.395625178043, 197.691713257043, 198.878700508003, 195.004975834965, 196.189215910647, 197.174064395495, 198.158912880344]
data = set2

In [6]:
xs = [i + 1 for i in range(len(data))]
num = range(len(xs))

In [7]:
for i in range(-5, 5):
    print(i)

-5
-4
-3
-2
-1
0
1
2
3
4


In [8]:
beta = [162.5, 0.03, 0.004]
print(cost(ys, xs, beta))

NameError: name 'ys' is not defined

In [15]:
search = []
for i in range(4):
    for j in range(4):
        for k in range(4):
            search.append("{} {} {}".format(i, j, k))
search = set(search)
search = [np.array([int(c) for c in s.split(" ")]) for s in search]
search

[array([2, 0, 0]),
 array([1, 0, 1]),
 array([1, 2, 3]),
 array([2, 2, 1]),
 array([3, 3, 0]),
 array([1, 1, 2]),
 array([3, 1, 2]),
 array([2, 2, 0]),
 array([3, 0, 2]),
 array([0, 3, 3]),
 array([0, 2, 2]),
 array([0, 0, 3]),
 array([2, 3, 2]),
 array([0, 1, 0]),
 array([0, 1, 2]),
 array([2, 0, 3]),
 array([3, 2, 3]),
 array([2, 2, 2]),
 array([2, 0, 1]),
 array([0, 2, 0]),
 array([0, 1, 1]),
 array([0, 0, 2]),
 array([0, 0, 0]),
 array([2, 2, 3]),
 array([2, 1, 3]),
 array([3, 0, 1]),
 array([3, 0, 0]),
 array([3, 2, 2]),
 array([1, 1, 0]),
 array([0, 2, 3]),
 array([0, 0, 1]),
 array([1, 3, 0]),
 array([0, 2, 1]),
 array([2, 1, 0]),
 array([3, 1, 0]),
 array([3, 2, 1]),
 array([0, 3, 0]),
 array([1, 0, 0]),
 array([2, 1, 2]),
 array([3, 3, 2]),
 array([1, 1, 3]),
 array([3, 3, 3]),
 array([2, 0, 2]),
 array([3, 1, 3]),
 array([2, 3, 3]),
 array([1, 2, 0]),
 array([3, 3, 1]),
 array([1, 1, 1]),
 array([3, 1, 1]),
 array([0, 3, 1]),
 array([1, 0, 3]),
 array([2, 1, 1]),
 array([0, 1

In [16]:
def model_growth(vals, days, max_iter=50):
    beta = np.array([vals[0], np.log(vals[-1] / vals[0]) / np.log(days[-1] - days[0]) * 4 / 5, 0.002])

    for n in range(max_iter):
        jacobian = np.zeros((len(days), 3))
        for i in range(len(days)):
            row = costd(vals[i], days[i], beta)
            if i == len(days) - 1:
                row = row / 100
            for j in range(3):
                jacobian[i][j] = row[j]
        residuals = np.array([r_adj(vals[i], x, beta) for i, x in enumerate(days)])
        vec = np.dot(np.linalg.inv(np.dot(np.transpose(jacobian), jacobian)), np.transpose(jacobian))
        shift = np.dot(vec, residuals)
        betas = []
        for s in search:
            betas.append([cost(vals, days, beta - shift / 10.0 ** s), beta - shift / 10.0 ** s])
            betas.append([cost(vals, days, beta + shift / 10.0 ** s), beta + shift / 10.0 ** s])
        og_cost = cost(vals, days, beta)
        min_cost = og_cost
        for b in betas:
            if b[0] < min_cost:
                min_cost = b[0]
                beta = b[1]
        if og_cost == min_cost:
            break
    return min_cost, beta, f(days[-1], beta)

In [17]:
for i in range(3, len(data)):
    print(model_growth(data[:i], xs[:i])[2])

164.4801195830456
168.91148430756232
170.68441762829434
175.39626167612292
176.72253077851573
179.80967291536544
181.33005344184684
183.00237950650217
183.99533030208087
184.9016737016234
185.78763642205976
185.0679393773801
185.93014701476866
186.2168970120054
186.59787809214637
187.39633949663846
188.1527012893385
188.89438133497802
189.64938955018422
190.33923377975543
191.08844988283232
191.87651889173048
192.56735271251887
193.38416241123173
194.18227494393025
195.03392725940145
195.99170674753844
196.96307545417352
197.3669416134656
197.75567656242197
198.26266504626912


In [18]:
model_growth(data, xs, 1000)

(141.00791342239765,
 array([1.60151846e+02, 4.49718878e-02, 1.67824289e-03]),
 198.69475454500153)

In [19]:
for x in xs:
    print(-r(0, x, beta))

160.12197611419617
165.4762199897948
168.80993355346283
171.30046532443515
173.3236371052696
175.04884541259537
176.56725151487325
177.93365300493352
179.1836005358141
180.3414872013705
181.42478548639562
182.446444424206
183.41633007045894
184.3421340746911
185.22997007069057
186.08477845364138
186.91060903046025
187.7108232852506
188.4882422419443
189.24525660248418
189.98391015946387
190.70596391167976
191.41294600721335
192.10619111719657
192.78687181744974
193.4560238502102
194.11456664540478
194.76332013108419
195.40301861069375
196.03432230102868
196.65782698895194
197.2740721635549
197.8835479039185
198.4867007443238


In [21]:
def f1(x, b):
    return  b[0] * np.e ** (b[1] / x + b[2] * np.log(x))

def f1p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] / x + b[2] * np.log(x))
    grad[1] = (b[0] * np.e ** (b[1] / x + b[2] * np.log(x))) / x
    grad[2] = np.log(x) * b[0] * np.e ** (b[1] / x + b[2] * np.log(x))
    return grad

def f2(x, b):
    return  x / (b[0] * x + b[1] * x ** (1 / 2) - b[2])

def f2p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] =  - ((x / (b[0] * x + b[1] * x ** (1 / 2) - b[2])) ** 2)
    grad[1] =  - x ** (3 / 2) / ((b[0] * x + b[1] * x ** (1 / 2) - b[2])) ** 2
    grad[2] =  x / (b[0] * x + b[1] * x ** (1 / 2) - b[2]) ** 2
    return grad

def f3(x, b):
    return  x ** b[0] * np.e ** (b[1] * x ** b[2])

def f3p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x) * x ** b[0] * np.e ** (b[1] * x ** b[2])
    grad[1] = x ** (b[0] + b[2]) * np.e ** (b[1] * x ** b[2])
    grad[2] = np.log(x) * x * b[1] ** (b[0] + b[2]) * np.e ** (b[1] * x ** b[2])
    return grad

def f4(x, b):
    return  b[0] * np.e ** (b[1] * x ** 2 + b[2] * x)

def f4p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x ** 2 + b[2] * x)
    grad[1] = x ** 2 * b[0] * np.e ** (b[1] * x ** 2 + b[2] * x)
    grad[2] = x * b[0] * np.e ** (b[1] * x ** 2 + b[2] * x)
    return grad
######################################
def f5(x, b):
    return  b[0] * x ** 2 + b[1] * x + b[2]

def f5p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** 2
    grad[1] = x
    grad[2] = 1
    return grad
#######################################
def f6(x, b):
    return  b[0] * np.log(x + b[1]) - b[2]

def f6p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x + b[1])
    grad[1] = b[0] / (x + b[1])
    grad[2] = -1
    return grad
########################################
def f7(x, b):
    return  b[0] * np.log(x + b[1])

def f7p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x + b[1])
    grad[1] = b[0] / (x + b[1])
    grad[2] = 0
    return grad
#######################################
def f8(x, b):
    return  (b[0] * x + b[1]) ** (1 / 2)

def f8p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x * (b[0] * x + b[1]) ** (-1 / 2) / 2
    grad[1] = (b[0] * x + b[1]) ** (-1 / 2) / 2
    grad[2] = 0
    return grad
#######################################
def f9(x, b):
    return  np.log(b[0] * np.e ** (b[1] * x) + 1) + b[2]

def f9p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x) / np.log(b[0] * np.e ** (b[1] * x) + 1)
    grad[1] = b[0] * x * e ** (b[1] * x) / np.log(b[0] * np.e ** (b[1] * x) + 1)
    grad[2] = 1
    return grad
#######################################
def f10(x, b):
    return  b[0] * x + b[1] / x ** 2 + b[2]

def f10p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = 1 / x ** 2
    grad[2] = 1
    return grad
#######################################
def f11(x, b):
    return  b[0] * x ** b[1] + b[2]

def f11p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** b[1]
    grad[1] = b[0] * np.log(x) * x ** b[1]
    grad[2] = 1
    return grad
#######################################
def f12(x, b):
    return  b[0] * np.e ** (-b[1] / (x + b[2]))

def f12p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (-b[1] / (x + b[2]))
    grad[1] = b[0] * (-1 / (x + b[2]))
    grad[2] = b[0] * (-b[1]) / (x + b[2]) ** 2 * np.e ** (-b[1] / (x + b[2]))
    return grad
#######################################
def f13(x, b):
    return  b[0] * x + b[1] * x ** (1 / 2) + b[2]

def f13p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = x ** (1 / 2)
    grad[2] = 1
    return grad
#######################################
def f14(x, b):
    return  b[0] * x + b[1] * x / (x - b[2])

def f14p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = x / (x - b[2])
    grad[2] = b[1] * x / (x - b[2]) ** 2
    return grad
########################################
def f15(x, b):
    return  b[0] * x + b[1] + b[2] / x

def f15p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = 1
    grad[2] = 1 / x
    return grad
########################################
def f16(x, b):
    return  b[0] * x + b[1]

def f16p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x
    grad[1] = 1
    grad[2] = 0
    return grad
#######################################
def f17(x, b):
    return  b[0] * np.e ** (b[1] * x) - b[2]

def f17p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x)
    grad[1] = b[0] * x * np.e ** (b[1] * x) 
    grad[2] = -1
    return grad
########################################
def f18(x, b):
    return  b[0] * x ** b[1] * np.e ** (b[2] * x)

def f18p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** b[1] * np.e ** (b[2] * x)
    grad[1] = b[0] * x * np.log(x) ** b[1] * np.e ** (b[2] * x)
    grad[2] = b[0] * x ** (b[1] + 1) * np.e ** (b[2] * x)
    return grad
########################################
def f19(x, b):
    return  b[0] * np.e ** (b[1] * x)

def f19p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x)
    grad[1] = b[0] * x * np.e ** (b[1] * x)
    grad[2] = 0
    return grad
#########################################
def f20(x, b):
    return  b[0] * x ** 2 + b[1] * x ** (1 / 2) + b[2]

def f20p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** 2
    grad[1] = x ** (1 / 2)
    grad[2] = 1
    return grad
########################################
def f21(x, b):
    return  b[0] * (np.e ** (b[1] * x) + 1)

def f21p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x) + 1
    grad[1] = b[0] * x * np.e ** (b[1] * x)
    grad[2] = 0
    return grad
#########################################
def f22(x, b):
    return  b[0] * x ** (1 / 2) + b[1]

def f22p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** (1 / 2)
    grad[1] = 1
    grad[2] = 0
    return grad
#########################################
def f23(x, b):
    return  b[0] - b[1] / x + b[2] / x ** 2

def f23p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = 1
    grad[1] = -1 / x
    grad[2] = 1 / x ** 2
    return grad
#########################################
def f24(x, b):
    return  b[0] * x ** (b[1] * x)

def f24p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x ** (b[1] * x)
    grad[1] = b[0] * x ** 2 * np.log(x) * x ** (b[1] * x)
    grad[2] = 0
    return grad
#########################################
def f25(x, b):
    return  (b[0] * x + b[1]) ** b[2]

def f25p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = x * b[2] * (b[0] * x + b[1]) ** (b[2] - 1)
    grad[1] = b[2] * (b[0] * x + b[1]) ** (b[2] - 1)
    grad[2] = np.log(b[0] * x + b[1]) * (b[0] * x + b[1]) ** b[2]
    return grad
#########################################
def f26(x, b):
    return  1 / (b[0] * x ** b[1] - b[2])

def f26p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = -x ** b[1] / (b[0] * x ** b[1] - b[2]) ** 2
    grad[1] = -b[0] * np.log(x) * x ** b[1] / (b[0] * x ** b[1] - b[2]) ** 2
    grad[2] = 1 / (b[0] * x ** b[1] - b[2]) ** 2
    return grad
########################################
def f27(x, b):
    return  b[0] * np.e ** (b[1] * x ** 2) - b[2]

def f27p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x ** 2)
    grad[1] = b[0] * x ** 2 * np.e ** (b[1] * x ** 2)
    grad[2] = -1
    return grad
########################################
def f28(x, b):
    return  b[0] * np.e ** (b[1] * x ** 2)

def f28p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.e ** (b[1] * x ** 2)
    grad[1] = b[0] * x ** 2 * np.e ** (b[1] * x ** 2)
    grad[2] = 0
    return grad
########################################
def f29(x, b):
    return  b[0] * np.log(x) + b[1]

def f29p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(x)
    grad[1] = 1
    grad[2] = 0
    return grad
########################################
def f30(x, b):
    return  b[0] * np.log(b[1] * np.log(x + b[2]))

def f30p(x, b):
    grad = [0 for _ in range(3)]
    grad[0] = np.log(b[1] * np.log(x + b[2]))
    grad[1] = b[0] * np.log(x + b[2]) / (b[1] * np.log(x + b[2]))
    grad[2] = b[0] / np.log(x + b[2]) ** 2
    return grad
########################################

In [22]:
funs = [
    (f1, f1p),
    (f2, f2p),
    (f3, f3p),
    (f4, f4p),
    (f5, f5p),
    (f6, f6p),
    (f7, f7p),
    (f8, f8p),
    (f9, f9p),
    (f10, f10p),
    (f11, f11p),
    (f12, f12p),
    (f13, f13p),
    (f14, f14p),
    (f15, f15p),
    (f16, f16p),
    (f17, f17p),
    (f18, f18p),
    (f19, f19p),
    (f20, f20p),
    (f21, f21p),
    (f22, f22p),
    (f23, f23p),
    (f24, f24p),
    (f25, f25p),
    (f26, f26p),
    (f27, f27p),
    (f28, f28p),
    (f29, f29p),
    (f30, f30p),
]

In [23]:
funs

[(<function __main__.f1(x, b)>, <function __main__.f1p(x, b)>),
 (<function __main__.f2(x, b)>, <function __main__.f2p(x, b)>),
 (<function __main__.f3(x, b)>, <function __main__.f3p(x, b)>),
 (<function __main__.f4(x, b)>, <function __main__.f4p(x, b)>),
 (<function __main__.f5(x, b)>, <function __main__.f5p(x, b)>),
 (<function __main__.f6(x, b)>, <function __main__.f6p(x, b)>),
 (<function __main__.f7(x, b)>, <function __main__.f7p(x, b)>),
 (<function __main__.f8(x, b)>, <function __main__.f8p(x, b)>),
 (<function __main__.f9(x, b)>, <function __main__.f9p(x, b)>),
 (<function __main__.f10(x, b)>, <function __main__.f10p(x, b)>),
 (<function __main__.f11(x, b)>, <function __main__.f11p(x, b)>),
 (<function __main__.f12(x, b)>, <function __main__.f12p(x, b)>),
 (<function __main__.f13(x, b)>, <function __main__.f13p(x, b)>),
 (<function __main__.f14(x, b)>, <function __main__.f14p(x, b)>),
 (<function __main__.f15(x, b)>, <function __main__.f15p(x, b)>),
 (<function __main__.f16(x, 